In [1]:
import gc
import os
import time

import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

from glob import glob
from tqdm import tqdm
from keras import backend as K
from keras.callbacks import ModelCheckpoint, CSVLogger, EarlyStopping
from keras.preprocessing import image                  
from sklearn.utils import shuffle
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dropout, Flatten, Dense
from keras.models import Sequential, Model
from keras.layers import BatchNormalization
from keras import regularizers, applications, optimizers, initializers
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0, EfficientNetB1, EfficientNetB2, EfficientNetB3, EfficientNetB4, EfficientNetB5, EfficientNetB6, EfficientNetB7

In [2]:
from pandas import DataFrame

class MultiLabelCrossEntropy:
    """
    Weight Cross Entropy Loss.
    
    For each class we will found two weights corresponding to the positive and negative frequency of our sample data.
    These weights will manage the way we update our network. And this has for objectives to manage the unbalanced class issue.
    """
        
    def __init__(self, labels : DataFrame, epsilon = 1e-7):
        
        self.epsilon = epsilon
        
        # Get the size of the data
        self.N = labels.shape[0]
        
        # Get the frequency occurence for each class
        self.freq_pos = np.sum(labels == 1, axis=0) / self.N
        self.freq_neg = np.sum(labels == 0, axis=0) / self.N
        
        # Set the loss weights for each labels 
        self.pos_weights = self.freq_neg
        self.neg_weights = self.freq_pos
        
    def contribution(self):
        """
        Get the weights' contribution for each labels.
        
        Returns :
            - double : Positive contribution
            - double : Negative contribution
        """
        return self.freq_pos * self.pos_weights, self.freq_neg * self.neg_weights
        
        
    def loss(self, y_true, y_pred):
        """
        Return weighted loss value. 
        """
        # Initialize loss to zero
        loss = 0.0
        
        for i in range(len(self.pos_weights)):
            loss += (
                -1 * K.mean(self.pos_weights[i] * y_true[:,i] * K.log(y_pred[:,i] + self.epsilon))
                    ) + (
                -1 * K.mean(self.neg_weights[i] * (1 - y_true[:,i]) * K.log(1 - y_pred[:,i] + self.epsilon))
                    )
        return loss

In [2]:
def binary_accuracy(y_true, y_pred):
    return K.mean(K.equal(y_true, K.round(y_pred)))

def precision_threshold(threshold = 0.5):
    def precision(y_true, y_pred):
        threshold_value = threshold
        y_pred = K.cast(K.greater(K.clip(y_pred, 0, 1), threshold_value), K.floatx())
        true_positives = K.round(K.sum(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(y_pred)
        precision_ratio = true_positives / (predicted_positives + K.epsilon())
        return precision_ratio
    return precision

def recall_threshold(threshold = 0.5):
    def recall(y_true, y_pred):
        threshold_value = threshold
        y_pred = K.cast(K.greater(K.clip(y_pred, 0, 1), threshold_value), K.floatx())
        true_positives = K.round(K.sum(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.clip(y_true, 0, 1))
        recall_ratio = true_positives / (possible_positives + K.epsilon())
        return recall_ratio
    return recall

def fbeta_score_threshold(beta = 1, threshold = 0.5):
    def fbeta_score(y_true, y_pred):
        threshold_value = threshold
        beta_value = beta
        p = precision_threshold(threshold_value)(y_true, y_pred)
        r = recall_threshold(threshold_value)(y_true, y_pred)
        bb = beta_value ** 2
        fbeta_score = (1 + bb) * (p * r) / (bb * p + r + K.epsilon())
        return fbeta_score
    return fbeta_score

def calculate_cm(y_true, y_pred):
    fp = np.sum((y_pred == 1) & (y_true == 0))
    tp = np.sum((y_pred == 1) & (y_true == 1))
    fn = np.sum((y_pred == 0) & (y_true == 1))
    tn = np.sum((y_pred == 0) & (y_true == 0))
    return tp, fp, fn, tn

def calculate_recall(tp, fp, fn, tn):
    return (tp)/(tp + fn)

def calculate_fallout(tp, fp, fn, tn):
    return (fp)/(fp + tn)

def calculate_fpr_tpr(y_true, y_pred):
    tp, fp, fn, tn = calculate_cm(y_true, y_pred)
    tpr = calculate_recall(tp, fp, fn, tn)
    fpr = calculate_fallout(tp, fp, fn, tn)
    return fpr, tpr

In [3]:
log = CSVLogger('saved_models/log_pretrained_CNN.csv')
checkpointer = ModelCheckpoint(filepath='saved_models/pretrainedVGG.best.from_scratch.hdf5', verbose=1, save_best_only=True)

In [4]:
diseases = [
    'Cardiomegaly','Emphysema','Effusion',
    'Hernia','Nodule','Pneumothorax',
    'Atelectasis','Pleural_Thickening',
    'Mass','Edema','Consolidation',
    'Infiltration','Fibrosis','Pneumonia'
]

dataset_df = pd.read_csv('/kaggle/input/data/Data_Entry_2017.csv')
# dataset_df = pd.read_csv('./dataset_information/Data_Entry_2017.csv')

In [5]:
# Applying One Hot Encoding to Labels
for disease in diseases:
    dataset_df[disease] = dataset_df['Finding Labels'].apply(lambda x: 1 if disease in x else 0)

In [6]:
image_labels = dataset_df[diseases].to_numpy()
image_paths = {
    os.path.basename(x): x for x in glob(os.path.join('..', 'input', 'data', 'images*', 'images', '*.png'))
#     os.path.basename(x): x for x in glob(os.path.join('.', 'images', '*.png'))
}

print(f"Samples Found: {len(image_paths)}")

Samples Found: 112120


In [7]:
# Storing path to each image against image name in the dataframe
dataset_df['Image Path'] = dataset_df['Image Index'].map(image_paths.get)

In [8]:
images_list = dataset_df['Image Path'].tolist()

labelB = (dataset_df[diseases].sum(axis = 1) > 0).tolist()
labelB = np.array(labelB, dtype = int)

In [9]:
def read_image_to_tensor(path, shape):
    # Loads RGB image to PIL format
    img = image.load_img(path, target_size = shape)
    
    # Convert PIL image to 3D tensor of specific shape
    # and normalizes it by dividing each pixel by 255
    normalized_image_tensor = image.img_to_array(img) / 255
    
    # Convert 3D tensor to 4D tensor with specific shape 
    # (1, shape, 3) and return it
    return np.expand_dims(normalized_image_tensor, axis = 0)

In [10]:
def image_to_array(paths, shape):
    images_arrays = [read_image_to_tensor(path, shape) for path in tqdm(paths, desc = "Progress", ncols = 100)]
    return np.vstack(images_arrays)

In [11]:
# Defining some hyper-parameters
IMAGE_SHAPE = (70, 70)
EPOCHS = 10
BATCH_SIZE = 32

In [12]:
# Samples in Training Set: 70%
# Samples in Validation Set: 10%

# Storing labels of samples for each set
train_labels = labelB[ : 78484][ : , np.newaxis]
valid_labels = labelB[78484 : 89696][ : , np.newaxis]

# Storing arrays of samples for each set
training_samples = image_to_array(images_list[ : 78484], shape = IMAGE_SHAPE)
validation_samples = image_to_array(images_list[78484 : 89696], shape = IMAGE_SHAPE)

Progress: 100%|███████████████████████████████████████████████| 11212/11212 [04:11<00:00, 44.66it/s]


In [13]:
# Creating a model with EfficientNet as base.
e_net = EfficientNetB2(
    weights = 'imagenet',
    include_top = False,
    input_shape = training_samples.shape[1 : ]
)

custom_classifier = Sequential()
custom_classifier.add(GlobalAveragePooling2D(input_shape = e_net.output_shape[1 : ]))
custom_classifier.add(Dropout(0.2))
custom_classifier.add(Dense(256, activation = 'relu'))
custom_classifier.add(Dropout(0.2))
custom_classifier.add(Dense(512, activation = 'relu'))
custom_classifier.add(Dropout(0.2))
custom_classifier.add(Dense(256, activation = 'relu'))
custom_classifier.add(Dropout(0.2))
custom_classifier.add(Dense(50, activation = 'relu'))
custom_classifier.add(Dropout(0.2))
custom_classifier.add(Dense(1, activation = 'sigmoid'))

model = Model(inputs = e_net.input, outputs = custom_classifier(e_net.output))
# model.summary()

2022-12-08 20:20:56.347971: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-08 20:20:56.445161: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-08 20:20:56.445965: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-08 20:20:56.447483: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

31801344/31790344 [==============================] - 3s 0us/step


In [14]:
# Defining 2 optimizers to test the model with.

SGD_optimizer = tf.keras.optimizers.SGD(
    learning_rate = 1e-4, 
    decay = 1e-6, 
    momentum = 0.9, 
    nesterov = True
)
adam_optimizer = tf.keras.optimizers.Adam(
    learning_rate = 0.001,
    beta_1 = 0.9,
    beta_2 = 0.999,
)

In [15]:
# Defining object for augmentation 

train_datagen = ImageDataGenerator(
    featurewise_center = False,  # set input mean to 0 over the dataset
    samplewise_center = False,  # set each sample mean to 0
    featurewise_std_normalization = False,  # divide inputs by std of the dataset
    samplewise_std_normalization = False,  # divide each input by its std
    zca_whitening = False,  # apply ZCA whitening
    rotation_range = 10,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range = 0.1,  # randomly shift images horizontally (fraction of total width)
    height_shift_range = 0.1,  # randomly shift images vertically (fraction of total height)
    horizontal_flip = True,  # randomly flip images
    vertical_flip = False 
)

In [16]:
# Compiling model with loss function, optimizer and metrics

model.compile(
    optimizer = SGD_optimizer,
    loss = 'binary_crossentropy',
    metrics = [
        'accuracy',
        tf.keras.metrics.FalseNegatives(),
        tf.keras.metrics.TrueNegatives(),
        tf.keras.metrics.FalsePositives(),
        tf.keras.metrics.TrueNegatives(),
        precision_threshold(threshold = 0.5), 
        recall_threshold(threshold = 0.5), 
        fbeta_score_threshold(beta=0.5, threshold = 0.5)
    ]
)

In [17]:
%%timeit -n1 -r1

history = model.fit_generator(
    train_datagen.flow(
        training_samples, 
        train_labels, 
        batch_size = BATCH_SIZE
    ),
    steps_per_epoch = len(training_samples) // BATCH_SIZE,
    validation_data = (validation_samples, valid_labels),
    validation_steps = len(validation_samples) // BATCH_SIZE,
    epochs = EPOCHS,
    callbacks = [checkpointer], 
    verbose = 1
)

/opt/conda/lib/python3.7/site-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
2022-12-08 20:21:04.768748: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/10


2022-12-08 20:21:17.743720: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


2452/2452 [==============================] - ETA: 0s - loss: 0.6949 - accuracy: 0.5282 - false_negatives: 25158.0000 - true_negatives: 31242.0000 - false_positives: 11856.0000 - true_negatives_1: 31242.0000 - precision: 0.4629 - recall: 0.2880 - fbeta_score: 0.3859

2022-12-08 20:25:40.969926: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 659265600 exceeds 10% of free system memory.


2452/2452 [==============================] - 289s 110ms/step - loss: 0.6949 - accuracy: 0.5282 - false_negatives: 25158.0000 - true_negatives: 31242.0000 - false_positives: 11856.0000 - true_negatives_1: 31242.0000 - precision: 0.4629 - recall: 0.2880 - fbeta_score: 0.3859 - val_loss: 0.7033 - val_accuracy: 0.4760 - val_false_negatives: 5836.0000 - val_true_negatives: 5282.0000 - val_false_positives: 39.0000 - val_true_negatives_1: 5282.0000 - val_precision: 0.1456 - val_recall: 0.0101 - val_fbeta_score: 0.0381

Epoch 00001: val_loss improved from inf to 0.70331, saving model to saved_models/pretrainedVGG.best.from_scratch.hdf5


/opt/conda/lib/python3.7/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 2/10
2452/2452 [==============================] - 258s 105ms/step - loss: 0.6878 - accuracy: 0.5436 - false_negatives: 28311.0000 - true_negatives: 35596.0000 - false_positives: 7496.0000 - true_negatives_1: 35596.0000 - precision: 0.4898 - recall: 0.1997 - fbeta_score: 0.3619
Epoch 3/10
2452/2452 [==============================] - 257s 105ms/step - loss: 0.6831 - accuracy: 0.5550 - false_negatives: 28210.0000 - true_negatives: 36396.0000 - false_positives: 6699.0000 - true_negatives_1: 36396.0000 - precision: 0.5158 - recall: 0.2029 - fbeta_score: 0.3776
Epoch 4/10
2452/2452 [==============================] - 255s 104ms/step - loss: 0.6784 - accuracy: 0.5638 - false_negatives: 26709.0000 - true_negatives: 35585.0000 - false_positives: 7511.0000 - true_negatives_1: 35585.0000 - precision: 0.5359 - recall: 0.2452 - fbeta_score: 0.4183
Epoch 5/10
2452/2452 [==============================] - 255s 104ms/step - loss: 0.6727 - accuracy: 0.5813 - false_negatives: 23087.0000 - true_negat

In [18]:
# Freeing Memory with Python's Garbase Collector
del training_samples
del validation_samples
del train_labels
del valid_labels
gc.collect()

980

In [19]:
# Samples in Testing Set: 20%
test_labels = labelB[89696 : ][ : , np.newaxis]
test_samples = image_to_array(images_list[89696 : ], shape = IMAGE_SHAPE)

prediction = model.predict(test_samples)

Progress: 100%|███████████████████████████████████████████████| 22424/22424 [09:11<00:00, 40.68it/s]
2022-12-08 21:14:43.933958: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1318531200 exceeds 10% of free system memory.


In [20]:
# Evaluation of Trained Model
testing_predictions = []
for val in prediction:
    if val[0] < 0.5:
        testing_predictions.append([0])
    else:
        testing_predictions.append([1])

testing_predictions = np.array(testing_predictions)

TP, FP, FN, TN = calculate_cm(test_labels, testing_predictions)
FPR, TPR = calculate_fpr_tpr(test_labels, testing_predictions)

threshold = 0.5
beta = 0.5

accuracy = K.eval(binary_accuracy(K.variable(value=test_labels), K.variable(value=prediction)))
precision = K.eval(precision_threshold(threshold = threshold)(K.variable(value=test_labels),K.variable(value=prediction)))
recall = K.eval(recall_threshold(threshold = threshold)(K.variable(value=test_labels),K.variable(value=prediction)))
f1_score = K.eval(fbeta_score_threshold(beta = beta, threshold = threshold)(K.variable(value=test_labels),K.variable(value=prediction)))

print (f"Accuracy: {accuracy} \nRecall: {recall} \nSpecificity: {TN / (TN + FP)}\nPrecision: {precision} \nF1-Score: {f1_score}\n")
print (f"True Positives: {TP} \nFalse Positives: {FP} \nFalse Negatives: {FN} \nTrue Negatives: {TN}\n")
print (f"False Positve Rate: {FPR * 100} % \nTrue Positive Rate: {TPR * 100} %")

Accuracy: 0.5903941988945007 
Recall: 0.8306812644004822 
Specificity: 0.37899237153156173
Precision: 0.5406176447868347 
F1-Score: 0.5812076926231384

True Positives: 8718 
False Positives: 7408 
False Negatives: 1777 
True Negatives: 4521

False Positve Rate: 62.10076284684383 % 
True Positive Rate: 83.06812767984755 %
